In [11]:
liked_books = ["2657", "6066095", "33246653" ]

In [ ]:
#find all users that liked the same books as us, then find all the ones that they liked

In [1]:
!head book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [14]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [15]:
len(csv_book_mapping)

2360651

In [ ]:
#find users who like the same books as us

In [16]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        if user_id in overlap_users:
            continue
        try:
            rating = int(rating)
        except ValueError:
            continue
        book_id = csv_book_mapping[csv_id]
        if book_id in liked_books and rating >=4:
            overlap_users.add(user_id)

In [17]:
rec_lines = []
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")

        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating]) #books that users who've liked the same books as us have read

In [18]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id","book_id","rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [22]:
top_recs = recs["book_id"].value_counts().head(10) #counts how many times book id has occured and gets the most common ones
top_recs = top_recs.index.values
#list of top book ids

In [23]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [ ]:
#right now we have book ids, want the titles

In [26]:
books_titles[books_titles["book_id"].isin(top_recs)] #gets all the book titles where the book ids is in top recs

,book_id,title,ratings,url,cover_image,mod_title
140511,1885,Pride and Prejudice,2078406,https://www.goodreads.com/book/show/1885.Pride...,https://images.gr-assets.com/books/1320399351m...,pride and prejudice
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
323031,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
396828,7624,Lord of the Flies,1638289,https://www.goodreads.com/book/show/7624.Lord_...,https://images.gr-assets.com/books/1327869409m...,lord of the flies
401395,5107,The Catcher in the Rye,2086945,https://www.goodreads.com/book/show/5107.The_C...,https://images.gr-assets.com/books/1398034300m...,the catcher in the rye
463463,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
615314,5470,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
770177,7613,Animal Farm,1928931,https://www.goodreads.com/book/show/7613.Anima...,https://images.gr-assets.com/books/1424037542m...,animal farm
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...


In [27]:
all_recs = recs["book_id"].value_counts()
all_recs

2657        159358
4671         99261
3            96889
2767052      90826
5107         87534
             ...  
25656290         1
24405064         1
23814457         1
25964295         1
12273026         1
Name: book_id, Length: 1661180, dtype: int64

In [28]:
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]

In [29]:
all_recs.head(5)

,book_id,book_count
0,2657,159358
1,4671,99261
2,3,96889
3,2767052,90826
4,5107,87534


In [30]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")


In [31]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])
#normalize the score based on how popular it is so our result is not overpopulized with popular books

In [32]:
all_recs.sort_values("score", ascending=False).head(10)


,book_id,book_count,title,ratings,url,cover_image,mod_title,score
5649,24909347,1418,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,24521.024390
11552,26856502,745,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,15857.857143
6513,28170940,1256,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,14882.415094
17395,31520883,504,"Untitled (An Ember in the Ashes, #4)",22,https://www.goodreads.com/book/show/31520883-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled an ember in the ashes 4,11546.181818
9335,29749098,906,"Catwoman: Soulstealer (DC Icons, #3)",73,https://www.goodreads.com/book/show/29749098-c...,https://s.gr-assets.com/assets/nophoto/book/11...,catwoman soulstealer dc icons 3,11244.328767
7016,30809786,1178,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,https://www.goodreads.com/book/show/30809786-a...,https://images.gr-assets.com/books/1507476834m...,a reaper at the gates an ember in the ashes 3,11191.000000
19398,36307629,452,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,9286.545455
0,2657,159358,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird,7800.593381
11301,33645809,759,"Legendary (Caraval, #2)",76,https://www.goodreads.com/book/show/33645809-l...,https://images.gr-assets.com/books/1506803078m...,legendary caraval 2,7580.013158
2515,33590260,2924,"Untitled (Throne of Glass, #7)",1190,https://www.goodreads.com/book/show/33590260-u...,https://images.gr-assets.com/books/1488914165m...,untitled throne of glass 7,7184.685714


In [33]:
all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False).head(10)


,book_id,book_count,title,ratings,url,cover_image,mod_title,score
5649,24909347,1418,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,24521.024390
11552,26856502,745,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,15857.857143
6513,28170940,1256,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,14882.415094
17395,31520883,504,"Untitled (An Ember in the Ashes, #4)",22,https://www.goodreads.com/book/show/31520883-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled an ember in the ashes 4,11546.181818
9335,29749098,906,"Catwoman: Soulstealer (DC Icons, #3)",73,https://www.goodreads.com/book/show/29749098-c...,https://s.gr-assets.com/assets/nophoto/book/11...,catwoman soulstealer dc icons 3,11244.328767
7016,30809786,1178,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,https://www.goodreads.com/book/show/30809786-a...,https://images.gr-assets.com/books/1507476834m...,a reaper at the gates an ember in the ashes 3,11191.000000
19398,36307629,452,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,9286.545455
0,2657,159358,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird,7800.593381
11301,33645809,759,"Legendary (Caraval, #2)",76,https://www.goodreads.com/book/show/33645809-l...,https://images.gr-assets.com/books/1506803078m...,legendary caraval 2,7580.013158
2515,33590260,2924,"Untitled (Throne of Glass, #7)",1190,https://www.goodreads.com/book/show/33590260-u...,https://images.gr-assets.com/books/1488914165m...,untitled throne of glass 7,7184.685714


In [34]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)


In [35]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
5649,24909347,1418,"Obsidio (The Illuminae Files, #3)",82,Goodreads,,obsidio the illuminae files 3,24521.024390
11552,26856502,745,"Vengeful (Villains, #2)",35,Goodreads,,vengeful villains 2,15857.857143
6513,28170940,1256,"Lethal White (Cormoran Strike, #4)",106,Goodreads,,lethal white cormoran strike 4,14882.415094
17395,31520883,504,"Untitled (An Ember in the Ashes, #4)",22,Goodreads,,untitled an ember in the ashes 4,11546.181818
9335,29749098,906,"Catwoman: Soulstealer (DC Icons, #3)",73,Goodreads,,catwoman soulstealer dc icons 3,11244.328767
7016,30809786,1178,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,Goodreads,,a reaper at the gates an ember in the ashes 3,11191.000000
19398,36307629,452,"King of Scars (King of Scars, #1)",22,Goodreads,,king of scars king of scars 1,9286.545455
11301,33645809,759,"Legendary (Caraval, #2)",76,Goodreads,,legendary caraval 2,7580.013158
2515,33590260,2924,"Untitled (Throne of Glass, #7)",1190,Goodreads,,untitled throne of glass 7,7184.685714
23590,32333338,369,Save the Date,19,Goodreads,,save the date,7166.368421
